# Rekurencyjne Odwracanie Mcierzy

## Analiza algorytmu

### Pseudokod rekurencyjnego algorytmu odwracania macierzy

```python
def inverse(A: Matrix) -> Matrix:

    n = size(A)     # rozmiar macierzy A

    # Warunek brzegowy - trywialna odwrotnoś
    if n == 1:
        return 1 / A

    # Podział macierzy A na bloki o jednakowym rozmiarze
    [A11 A12]
    [A21 A22] = A

    # Zmienne pomocnicze
    A11_inv = inverse(A11)
    S22 = A22 - A21 * A11_inv * A12
    S22_inv = inverse(S22)

    # Oblicznie bloków składających się na macierz wynikową B
    B11 = A11_inv + A11_inv * A12 * S22_inv * A21 * A11_inv
    B12 = -A11_inv * A12 * S22_inv
    B21 = -S22_inv * A21 * A11_inv
    B22 = S22_inv

    # Składanie obliczonych bloków w wynikową macierz B
    B = [B11, B12]
        [B21, B22]

    return B
```

### Pseudokod algorytmu rekurencyjnego mnożenia macierzy
```python
def multiply_strassen_with_classic(A: Matrix, B: Matrix, l: int = 6) -> Matrix:

    def strassen(A: Matrix, B: Matrix, n: int):

        # Warunek brzegowy - trywialne mnożenie
        if n == 1:
            return A * B

        # Warunek brzegowy - rozmiar macierzy suboptymalny aby wywoływać rekurencyjnie
        elif n <= l:
            return multiply_classic(A, B)
        
        # Podział macierzy A na bloki o jednakowym rozmiarze
        [A11 A12]
        [A21 A22] = A

        # Podział macierzy B na bloki o jednakowym rozmiarze
        [B11 B12]
        [B21 B22] = B

        # Wywołania rekurencyjne aby wyliczy macierze pomocnicze P_i, i z {1, 2, ..., 7}
        P1 = strassen(A11 + A22, B11 + B22, n // 2)
        P2 = strassen(A21 + A22, B11, n // 2)
        P3 = strassen(A11, B12 - B22, n // 2)
        P4 = strassen(A22, B21 - B11, n // 2)
        P5 = strassen(A11 + A12, B22, n // 2)
        P6 = strassen(A21 - A11, B11 + B12, n // 2)
        P7 = strassen(A12 - A22, B21 + B22, n // 2)

        # Składanie macierzy wynikowej C z bloków powstałych z P_i
        C11 = P1 + P4 - P5 + P7
        C12 = P3 + P5
        C21 = P2 + P4
        C22 = P1 - P2 + P3 + P6

        C = [C11, C12]
            [C21, C22]

        return C
    
    return strassen(A, B, size(A))
```

### FLO: liczba operacji zmiennoprzecinkowych w algorytmie

In [1]:
# Funkcja anonimowa obliczająca liczbę operacji dla k
flo = lambda k: pow(2, k) + sum([pow(2, k - i) * (46 * pow(7, i - 1) + pow(4, i - 1) - 36 * pow(2, i - 1) + 1) for i in range(2, k + 1)])

## Eksperymenty

In [4]:
from inverse import inverse
from utils import *
from timeit import default_timer as timer

SEED = 42
MATRICES_FILE_PATH = "matrices.dat"

In [6]:
np.random.seed(SEED)

matrices = [generate_invertible_matrix(2 ** k, 'I') for k in range(1, 9 + 1)]
save(MATRICES_FILE_PATH, matrices)

Generated 2x2 matrix!
Generated 4x4 matrix!
Generated 8x8 matrix!
Generated 16x16 matrix!
Generated 32x32 matrix!
Generated 64x64 matrix!
Keep searching...
Generated 128x128 matrix!
Keep searching...
Generated 256x256 matrix!
Keep searching...


In [ ]:
start = timer()



end = timer()
print(end - start)

### Czas wykonania

### Liczba operacji